In [3]:
from quspin.operators import hamiltonian# operators
from quspin.basis import boson_basis_1d # Hilbert space boson basis
from quspin.basis import tensor_basis,spinless_fermion_basis_1d # Hilbert spaces
from quspin.basis import spin_basis_1d # Hilbert space spin basis
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function, division
from quspin.tools.measurements import obs_vs_time # t_dep measurements
from quspin.tools.Floquet import Floquet_t_vec

In [12]:
sites=1
###### parameters
L_modes = sites # system size
L_spin = 1
total_boson_number = sites
cutoff = sites+2
h = 1 # field strength
t = 1
###### create the basis
# building the two bases to tensor together
basis_spin = spin_basis_1d(L=L_spin)
basis_boson=boson_basis_1d(L=L_modes,sps=cutoff,Nb=total_boson_number)
basis = tensor_basis(basis_spin,basis_boson)
# print(basis)

##### create model
hop=[[-t,i,i,i+1] for i in range(L_modes-1)]
field = [[-h,i] for i in range(L_spin)]
static=[["z|+-",hop],["z|-+",hop],["x|",field]]
###### setting up operators
# set up hamiltonian dictionary and observable (imbalance I)
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
H = hamiltonian(static,[],basis=basis,**no_checks)
# E, _ = H.eigsh(k=2)

In [19]:
##### define initial state #####
# define atom ground state
#psi_at_i=np.array([1.0,0.0]) # spin-down eigenstate of \sigma^z in QuSpin 0.2.3 or older
psi_at_i=np.array([0.0,1.0])  # spin-down eigenstate of \sigma^z in QuSpin 0.2.6 or newer
# set up initial bosonic state
psi_bn_i = np.zeros(cutoff,dtype=np.float64)
psi_bn_i[1] = 1.0
# compute atom-photon initial state as a tensor product
psi_i=np.kron(psi_at_i,psi_bn_i)

In [20]:
t=Floquet_t_vec(3,30)
psi_t=H.evolve(psi_i,t.i,t.vals,iterate=True,rtol=1E-9,atol=1E-9)

ValueError: v0 must have 2 elements

In [21]:
obs_args={"basis":basis,"check_herm":False,"check_symm":False}
n=hamiltonian([["|n", [[1.0  ]] ]],[],dtype=np.float64,**obs_args)
Obs_t = obs_vs_time(psi_t,t.vals,{"n":n})
O_n = Obs_t["n"]

/tmp/ipykernel_10337/1101071075.py:2: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_pcon=False in hamiltonian
  n=hamiltonian([["|n", [[1.0  ]] ]],[],dtype=np.float64,**obs_args)


ValueError: not enough indices for opstr in: |n, ()

In [ ]:
##### plot results #####
import matplotlib.pyplot as plt
import pylab
# define legend labels
str_n = "$\\langle n\\rangle,$"
# plot spin-photon data
fig = plt.figure()
plt.plot(t.vals/t.T,O_n/Nph,"k",linewidth=1,label=str_n)
# label axes
plt.xlabel("$t/T$",fontsize=18)
# set y axis limits
plt.ylim([-1.1,1.4])
# display legend horizontally
plt.legend(loc="upper right",ncol=5,columnspacing=0.6,numpoints=4)
# update axis font size
plt.tick_params(labelsize=16)
# turn on grid
plt.grid(True)
# save figure
plt.tight_layout()
plt.savefig('example3.pdf', bbox_inches='tight')
# show plot
#plt.show()
plt.close()